# Importing the necessary packages

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
# System packages
import os
import csv
from IPython.display import clear_output
import time
from dataclasses import dataclass
import copy

# Science Packages
import numpy as np
import scipy as sc
from scipy import io
import matplotlib.pyplot as plt

# Data Packages
import h5py
import json

# CAD Packages
import gdstk
import lib.GDSTKUtils as utils
import lib.dither_utils as dit
import lib.GDSTKMasks as mask_util

# Import Parameters
from lib.mask_dict import *


In [4]:
# default size unit is microns, so define handy shortcuts for other units
um = 1
mm = 1e3
cm = 1e4

In [5]:
@dataclass
class dither_obj(object):
    x: np.matrix
    y: np.matrix
    block_size: np.matrix
        

In [6]:
def invert_coords(x_arr, y_arr):
    tmp_x = y_arr
    tmp_y = x_arr
    return tmp_x, tmp_y

def convert_coords(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_y = dims - y_arr
        tmp_x = x_arr
    elif type(dims) == list:
        tmp_y = dims[0] - y_arr
        tmp_x = x_arr
    return tmp_x, tmp_y

def shift_to_center(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_x = x_arr - (dims/ 2)
        tmp_y = y_arr - (dims/ 2)
    elif type(dims) == list:
        tmp_x = x_arr - (dims[0]/ 2)
        tmp_y = y_arr - (dims[1]/ 2)
    return tmp_x, tmp_y

def shift_to_zero(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_x = x_arr + (dims/ 2)
        tmp_y = y_arr + (dims/ 2)
    elif type(dims) == list:
        tmp_x = x_arr + (dims[0]/ 2)
        tmp_y = y_arr + (dims[1]/ 2)
    return tmp_x, tmp_y

def merge_centers(led_real_pos, pat_center, dims, invert = True, index = 0):
    if invert == True:
        center_led = [(led_real_pos['x'][index, dims[1]-1] + led_real_pos['x'][index,0])/2, (led_real_pos['y'][0,index] + led_real_pos['y'][dims[0]-1, index])/2]
    elif invert == False:
        center_led = [(led_real_pos['x'][index, dims[1] -1] + led_real_pos['x'][index,0])/2, (led_real_pos['y'][dims[0]-1, index] + led_real_pos['y'][0, index])/2]
    shift = [pat_center[0] - center_led[0] , pat_center[1] - center_led[1]] 
    return shift

## Reading in data
### Matlab files

In [9]:
# Loading the Matrix
mat_path = os.path.join("Data", "Positions", "221026 Masks", "mnistspread_bleed_221025205444_cellarray.mat")
mat = io.loadmat(mat_path)

# Reading cell data
net0 = mat['net0cell']
net1 = mat['net1cell']
net2 = mat['net2cell']

### h5 files

In [10]:
# File locations
LED_input_file = os.path.join("Data", "Positions", "positions", "LED_positions_input.h5")
LED_dev1_file = os.path.join("Data", "Positions", "positions", "LED_positions_device1.h5")
LED_dev2_file = os.path.join("Data", "Positions", "positions", "LED_positions_device2.h5")

PD_dev1_file = os.path.join("Data", "Positions", "positions", "PD_positions_device_1.h5")
PD_dev2_file = os.path.join("Data", "Positions", "positions", "PD_positions_device_2.h5")
PD_output_file = os.path.join("Data", "Positions", "positions", "PD_positions_output_ideal.h5")

# Loading the h5 file variables
LED_input_h5 = h5py.File(LED_input_file, "r")
LED_dev1_h5 = h5py.File(LED_dev1_file, "r")
LED_dev2_h5 = h5py.File(LED_dev2_file, "r")

PD_output_h5 = h5py.File(PD_output_file, "r")
PD_dev1_h5 = h5py.File(PD_dev1_file, "r")
PD_dev2_h5 = h5py.File(PD_dev2_file, "r")

## Obtaining the data and transposing
# Setting empty dictionaries
LED_input_cen, LED_dev1_cen, LED_dev2_cen, PD_dev1_cen, PD_dev2_cen, PD_output_cen = {}, {}, {}, {}, {}, {}

# Input
LED_input_cen['x'] = np.array(LED_input_h5.get('LED_cen_x'))*mm
LED_input_cen['y'] = np.array(LED_input_h5.get('LED_cen_y'))*mm
LED_input_cen['x'], LED_input_cen['y'] = np.transpose(LED_input_cen['x']), np.transpose(LED_input_cen['y'])
LED_input_cen['x'], LED_input_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_input_cen['x'], LED_input_cen['y'])


# LED1
LED_dev1_cen['x'] = np.array(LED_dev1_h5.get('LED_cen_x'))*mm
LED_dev1_cen['y'] = np.array(LED_dev1_h5.get('LED_cen_y'))*mm
LED_dev1_cen['x'], LED_dev1_cen['y'] = np.transpose(LED_dev1_cen['x']), np.transpose(LED_dev1_cen['y'])
LED_dev1_cen['x'], LED_dev1_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_dev1_cen['x'], LED_dev1_cen['y'])

# LED2
LED_dev2_cen['x'] = np.array(LED_dev2_h5.get('LED_cen_x'))*mm
LED_dev2_cen['y'] = np.array(LED_dev2_h5.get('LED_cen_y'))*mm
LED_dev2_cen['x'], LED_dev2_cen['y'] = np.transpose(LED_dev2_cen['x']), np.transpose(LED_dev2_cen['y'])
LED_dev2_cen['x'], LED_dev2_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_dev2_cen['x'], LED_dev2_cen['y'])

# PD1 
PD_dev1_cen['x'] = np.array(PD_dev1_h5.get('PD_cen_x'))*mm
PD_dev1_cen['y'] = np.array(PD_dev1_h5.get('PD_cen_y'))*mm
PD_dev1_cen['x'], PD_dev1_cen['y'] = np.transpose(PD_dev1_cen['x']), np.transpose(PD_dev1_cen['y'])
PD_dev1_cen['x'], PD_dev1_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_dev1_cen['x'], PD_dev1_cen['y'])


# PD2
PD_dev2_cen['x'] = np.array(PD_dev2_h5.get('PD_cen_x'))*mm
PD_dev2_cen['y'] = np.array(PD_dev2_h5.get('PD_cen_y'))*mm
PD_dev2_cen['x'], PD_dev2_cen['y'] = np.transpose(PD_dev2_cen['x']), np.transpose(PD_dev2_cen['y'])
PD_dev2_cen['x'], PD_dev2_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_dev2_cen['x'], PD_dev2_cen['y'])

# Output
PD_output_cen['x'] = np.array(PD_output_h5.get('PD_cen_x'))*mm
PD_output_cen['y'] = np.array(PD_output_h5.get('PD_cen_y'))*mm
PD_output_cen['x'], PD_output_cen['y'] = np.transpose(PD_output_cen['x']), np.transpose(PD_output_cen['y'])
PD_output_cen['x'], PD_output_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_output_cen['x'], PD_output_cen['y'])

In [11]:
json_file = os.path.join('Data', 'JSON', 'MNIST 215.json')

# Mask sheet

In [24]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [25]:
# Creating a meshgrid of center positions for all the masks
pos_x_arr = np.arange(20*mm, 280*mm, 35*mm)
pos_y_arr = np.arange(27.5*mm, 205*mm, 35*mm)
center_x, center_y = np.meshgrid(pos_x_arr, pos_y_arr)

In [26]:
# Constructing the list with the dictionaries
mask_list = [
    weight_mask_184_01, weight_mask_184_01, weight_mask_184_01,
    weight_mask_184_12, weight_mask_184_12, weight_mask_184_12,
    weight_mask_184_23, weight_mask_184_23, weight_mask_184_23,
    weight_mask_200_01, weight_mask_200_01, weight_mask_200_01,
    weight_mask_200_12, weight_mask_200_12, weight_mask_200_12,
    weight_mask_200_23, weight_mask_200_23, weight_mask_200_23,
    weight_mask_216_01, weight_mask_216_01, weight_mask_216_01,
    weight_mask_216_01, weight_mask_216_12, weight_mask_216_12,
    weight_mask_216_01, weight_mask_216_23, weight_mask_216_23,
    cb_mask_01, cb_mask_01, cb_mask_01,
    cb_mask_12, cb_mask_12, cb_mask_12,
    cb_mask_23, cb_mask_23, cb_mask_23,
    ph_mask_01, ph_mask_01, ph_mask_01, ph_mask_01,
    ph_mask_shifted_01, ph_mask_shifted_01, ph_mask_shifted_01, ph_mask_shifted_01,
    ph_mask_shifted_12, ph_mask_shifted_12, ph_mask_shifted_12, ph_mask_shifted_12,
]

In [27]:
index = 0
for x, y in zip(center_x.flatten(), center_y.flatten()):
    # masks = mask_util.generate_weight_mask_from_json(masks, "0", json_file, [x, y], net0, LED_input_cen, PD_dev1_cen)
    masks = mask_util.sel_mask(masks, mask_list[index], [x,y])
    index += 1

In [28]:
# Writing the mask array to the GDS file
mask_array.write_gds('test.gds')

In [18]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [19]:
# masks = mask_util.sel_mask(masks, mask_list[0], [x,y])
masks = mask_util.chequerboard_array(masks, [0,0], 10, LED_input_cen, PD_dev1_cen, 'Test')

In [20]:
# Writing the mask array to the GDS file
mask_array.write_gds('test.gds')

In [17]:
# # Creating all the borders 
# for x, y in zip(center_x.flatten(), center_y.flatten()):
#     slide_border = mask_util.rect_border([x, y], [30.25*mm, 30.25*mm], 250*um)
#     masks.add(*slide_border)

## Storing data in JSON files
### Setting the different values for the different classes of masks

In [39]:
# Files for 185
mnist0_185 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (185um)",
    "magnification": 10,
    "dither_size": 185*um,
}

mnist1_185 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (185um)",
    "magnification": 12.5,
    "dither_size": 185*um,
}

mnist2_185 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (185um)",
    "magnification": 12.5,
    "dither_size": 185*um,
}

master_dict_185 = {
    '0': mnist0_185,
    '1': mnist1_185,
    '2': mnist2_185,
}

with open("MNIST 185.json", "w") as outfile:
    json.dump(master_dict_185, outfile)

In [40]:
# Files for 200
mnist0_200 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (200um)",
    "magnification": 10,
    "dither_size": 200*um,
}

mnist1_200 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (200um)",
    "magnification": 12.5,
    "dither_size": 200*um,
}

mnist2_200 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (200um)",
    "magnification": 12.5,
    "dither_size": 200*um,
}

master_dict_200 = {
    '0': mnist0_200,
    '1': mnist1_200,
    '2': mnist2_200,
}

with open("MNIST 200.json", "w") as outfile:
    json.dump(master_dict_200, outfile)

In [41]:
# Files for 225
mnist0_215 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (215um)",
    "magnification": 10,
    "dither_size": 215*um,
}

mnist1_215 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (215um)",
    "magnification": 12.5,
    "dither_size": 215*um,
}

mnist2_215 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (215um)",
    "magnification": 12.5,
    "dither_size": 215*um,
}

master_dict_215 = {
    '0': mnist0_215,
    '1': mnist1_215,
    '2': mnist2_215,
}

with open("MNIST 215.json", "w") as outfile:
    json.dump(master_dict_215, outfile)

# Individual Mask

### Chequerboard Mask

In [184]:
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDSPY library
gds_array = gdstk.Library()

# GDSPY initialization stuff
gds_mask = gds_array.new_cell("Block Cell")

In [185]:
led_real_pos = copy.deepcopy(LED_dev2_cen)
pd_real_pos = copy.deepcopy(PD_output_cen)
pd_real_pos['x'], pd_real_pos['y'] = utils.shift_to_center([25*mm, 25*mm], pd_real_pos['x'], pd_real_pos['y'])

dims = led_real_pos['x'].shape
pat_center = [15*mm, 15*mm]
magnification = 10
box_size = [150*um, 150*um]

center_led = [(led_real_pos['x'][dims[0]-1, dims[1]-1] + led_real_pos['x'][0,0])/2, (led_real_pos['y'][0,0] + led_real_pos['y'][dims[0]-1, dims[1]-1])/2]
shift = merge_centers(led_real_pos, pat_center, dims=dims)
shift_led = [-(1/magnification * (led_real_pos['x'] - center_led[0])) + shift[0], -(1/magnification * (led_real_pos['y'] - center_led[1])) + shift[1]]
center_pos = [shift_led[0] + led_real_pos['x'], shift_led[1] + led_real_pos['y']]
center_cb = [-(1/magnification) * pd_real_pos['x'], -(1/magnification) * pd_real_pos['y']]

tmat = sc.linalg.toeplitz(np.mod(np.arange(1,11,1), 2))

for li in range(0, led_real_pos['x'].shape[0]):
    for lj in range(0, led_real_pos['y'].shape[1]):
        border = mask_util.rect_border([center_pos[0][li,lj], center_pos[1][li,lj]], [2.25*mm, 2.25*mm], 250*um)
        gds_mask.add(*border)

        # Adding alignment markers between the spots
        marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-left
        marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-left
        marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-right
        marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-right
        gds_mask.add(marker_11, marker_12, marker_21, marker_22)

        center_cb_shift_x, center_cb_shift_y = center_cb[0] + center_pos[0][li,lj], center_cb[1] + center_pos[1][li,lj]
        for idx1 in range(0, center_cb_shift_x.shape[0]):
            for idx2 in range(0, center_cb_shift_x.shape[1]):
                if tmat[idx1, idx2] == 1:
                    rect = gdstk.rectangle(*utils.convert_rect(box_size, [center_cb_shift_x[idx1, idx2], center_cb_shift_y[idx1, idx2]]))
                    gds_mask.add(rect)

In [107]:
# Writing the mask array to the GDS file
gds_array.write_gds('test2.gds')

In [134]:
center_cb_shift_x.shape

(8, 8)

In [88]:
temp_x = center_cb[0]
temp_y = center_cb[1]

### My Code

In [63]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [64]:
# Creating the cut border 0f 30mm by 20mm around the mask
pat_center = [15*mm, 15*mm]
slide_border = mask_util.rect_border([pat_center[0], pat_center[1]], [30.25*mm, 30.25*mm], 250*um)
masks.add(*slide_border)

# Selecting the type of the mask
sel_cell = copy.deepcopy(net0)

# Smaller block sizes
inner_dims = sel_cell[0,0].shape

# Setting useful values
magnification = 10
dims = sel_cell.shape

In [65]:
# Setting the sart, step and end values of the ideal positions
ideal_start_pos, ideal_end_pos, ideal_sep = 1.25*mm, 25*mm, 2.5*mm

# Initializing an empty dictionary to hold ideal positions
pd_ideal_pos = {}
pd_real_pos = {}
led_real_pos = {}

# Generating a meshgrid of values of idela positions
pd_ideal_arr = np.arange(ideal_start_pos, ideal_end_pos, ideal_sep)
pd_ideal_pos['x'], pd_ideal_pos['y'] =  np.meshgrid(pd_ideal_arr, pd_ideal_arr)
pd_ideal_pos['x'], pd_ideal_pos['y'] = convert_coords(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])
pd_ideal_pos['x'], pd_ideal_pos['y'] = shift_to_center(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])


# Assigning the created variables
pd_real_pos = copy.deepcopy(PD_dev1_cen)
pd_real_pos['x'], pd_real_pos['y'] = shift_to_center(25*mm, pd_real_pos['x'], pd_real_pos['y'])
led_real_pos = copy.deepcopy(LED_input_cen)

# Inverting the coordinates (Not required since it's been done in the files shared by Alex)
# led_real_pos['x'], led_real_pos['y'] = convert_coords(25*mm, led_real_pos['x'], led_real_pos['y'])
# pd_real_pos['x'], pd_real_pos['y'] = convert_coords(25*mm, pd_real_pos['x'], pd_real_pos['y'])
# pd_real_pos['x'], pd_real_pos['y'] = shift_to_center(25*mm, pd_real_pos['x'], pd_real_pos['y'])

# Finding center position of the matrix
center_led = [(led_real_pos['x'][dims[0]-1, dims[1]-1] + led_real_pos['x'][0,0])/2, (led_real_pos['y'][0,0] + led_real_pos['y'][dims[0]-1, dims[1]-1])/2]

# Finding shifts
shift = utils.merge_centers(led_real_pos, pat_center, dims=dims)

In [66]:
for li in range(0, sel_cell.shape[0]):
    for lj in range(0, sel_cell.shape[1]):
        # Initializing an empty dictionary to hold coordinates and dimensions of the dithered values
        dither_mat = {}

        # Diagnostics
        # print('sum: {}'.format(str(li+lj)))
        # print('li: {} \t lj: {}'.format(str(li), str(lj)))
        
        # Dithering the values in the 8 by 8 matrix
        dither_mat['y'], dither_mat['x'], dither_mat['dim'] = dit.dither_16x(sel_cell[li,lj].flatten(), 0.0125) # Because the output from the dither_16x returns y dimension first
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = np.array(dither_mat['x'], dtype=object)*mm, np.array(dither_mat['y'], dtype=object)*mm, np.array(dither_mat['dim'], dtype=object)*mm
        
        # Solution for the problem with recasting a matrix of empty arrays. This creates a new matrix of empty arrays of suitable size instead of recasting
        if dither_mat['x'].size == 0:
            # print('empty')
            empty_mat = np.empty(inner_dims, dtype=object)
            for li_loop1 in range(0, empty_mat.shape[0]):
                for lj_loop1 in range(0, empty_mat.shape[1]):
                    empty_mat[li_loop1,lj_loop1] = np.array([])
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = empty_mat, empty_mat, empty_mat
        else:
            # Else, do the reshaping
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'].reshape(inner_dims), dither_mat['y'].reshape(inner_dims), dither_mat['dim'].reshape(inner_dims)
        
        # Shifting the positions of the 10 by 10 weight matrices based on the LED positions and the also shifted for all of the patterns to be coincidental on the next layer
        shift_led = [-(1/magnification * (led_real_pos['x'][li,lj] - center_led[0])) + shift[0], -(1/magnification * (led_real_pos['y'][li,lj] - center_led[1])) + shift[1]]
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'] + led_real_pos['x'][li,lj] + shift_led[0], dither_mat['y'] + led_real_pos['y'][li,lj] + shift_led[1], dither_mat['dim']
        
        # Debug test by placing a border where the masks are supposed to be
        # border = mask_util.rect_border([shift_led[0] + led_real_pos['x'][li,lj], shift_led[1] + led_real_pos['y'][li,lj]], [2.25*mm, 2.25*mm], 250*um)
        # masks.add(*border)

        # Adding alignment markers between the spots
        marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-left
        marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-left
        marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-right
        marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-right
        masks.add(marker_11, marker_12, marker_21, marker_22)

        # Looping within the 10 by 10 matrix 
        for idx1 in range(0, dither_mat['x'].shape[0]):
            for idx2 in range(0, dither_mat['x'].shape[1]):
                # print('idx1: {} \t idx2: {}'.format(str(idx1), str(idx2)))
                sel_mat = {}
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = dither_mat['x'][idx1, idx2], dither_mat['y'][idx1, idx2], dither_mat['dim'][idx1, idx2]
                if sel_mat['x'].size == 0 or sel_mat['y'].size == 0:
                    # Skipping if the matrix is empty. Happens for 8 by 8 row for input
                    continue

                # Shifting the matrix according to the real positions of the photodiodes so the shifts are accounted for
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = sel_mat['x'] - ((1/magnification) * pd_real_pos['x'][idx1, idx2]), sel_mat['y'] - ((1/magnification) * pd_real_pos['y'][idx1, idx2]), sel_mat['dim']
                
                # Placing rectangles at appropriate positions by looping through the dither blocks
                for idx3 in range(0, sel_mat['x'].shape[0]):
                    # print('idx1: {} \t idx2: {} \t idx3: {}'.format(str(idx1), str(idx2), str(idx3)))
                    pattern = gdstk.rectangle(*utils.convert_rect([sel_mat['dim'][idx3], sel_mat['dim'][idx3]], [sel_mat['x'][idx3], sel_mat['y'][idx3]]))
                    masks.add(pattern)

In [67]:
text_pos = (0.5*mm, 0.5*mm)
desc = gdstk.text(r'Input Text', 0.75*mm, text_pos)
masks.add(*desc)

In [68]:
# Writing the mask array to the GDS file
mask_array.write_gds('test.gds')

In [38]:
temp_x = LED_dev1_cen['x']
temp_y = LED_dev1_cen['y']

## Individual Calculations
### Dithering the input values

In [74]:
dither_mat = {}
tmp_mat = dit.dither_16x(net0[0,0].flatten(), 0.0125)
dither_mat['y'] = np.array(tmp_mat[0], dtype=object).reshape(10,10)*mm
dither_mat['x'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm
dither_mat['dim'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm

# Archival

In [7]:
# Creating a meshgrid of center positions
pos_x_arr = np.arange(20*mm, 280*mm, 35*mm)
pos_y_arr = np.arange(27.5*mm, 205*mm, 35*mm)
center_x, center_y = np.meshgrid(pos_x_arr, pos_y_arr)
zero_x, zero_y = shift_to_zero([25*mm, 25*mm], center_x, center_y)

## Creating the borders

In [8]:
# slide_border = mask_util.rect_border((center_x.flatten(), center_y.flatten()), [30.25*mm, 30.25*mm], 250*um)
for x, y in zip(center_x.flatten(), center_y.flatten()):
    # print("x: ", x, "\t y:", y)
    slide_border = mask_util.rect_border([x, y], [30.25*mm, 30.25*mm], 250*um)
    masks.add(*slide_border)

## Creating the alignment markers

In [10]:
for x,y in zip(LED_dev1_cen_x.flatten(), LED_dev1_cen_y.flatten()):
    # print('x: ', x, '\t y: ', y)
    marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [zero_x[0][0] + x - 1.5*mm, zero_y[0][0] + y + 1.5*mm])) # top-left
    marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [zero_x[0][0] + x - 1.5*mm, zero_y[0][0] + y - 1.5*mm])) # bottom-left
    marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [zero_x[0][0] + x + 1.5*mm, zero_y[0][0] + y + 1.5*mm])) # top-right
    marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [zero_x[0][0] + x + 1.5*mm, zero_y[0][0] + y - 1.5*mm])) # bottom-right
    masks.add(marker_11, marker_12, marker_21, marker_22)

NameError: name 'LED_dev1_cen_x' is not defined

In [ ]:
# Test location of the LEDs
for x1, y1 in zip(zero_x.flatten(), zero_y.flatten()):
    for x2,y2 in zip(LED_dev1_cen_x.flatten(), LED_dev1_cen_y.flatten()):
        # print('x: ', x, '\t y: ', y)
        marker_test = gdstk.rectangle(*utils.convert_rect([2*mm, 2*mm], [x1 + x2, y1 + y2]))
        masks.add(marker_test)

mask_array.write_gds('test.gds')

In [11]:
mask_array.write_gds('test.gds')

In [254]:
# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [255]:
# slide_border = utils.rect_border((center_x.flatten(), center_y.flatten()), [30.25*mm, 30.25*mm], 250*um)
x = 20*mm
y = 27.5*mm
slide_border = mask_util.rect_border([x, y], [30.25*mm, 30.25*mm], 250*um)
masks.add(*slide_border)

# Non-functional code

## Calculating Positions
Image that shows how the datastructure that holds all the data is designed. Each subsequent level shows what's contained within and then the dictionary keys or the shape of the numpy array for each level. The datatype is mentioned at the end  
<center><img src='Images/221102 Data Structure.jpg' width = 1000></center>

### Setting some values 

In [11]:
# Selecting the type of the mask
sel_mat = copy.deepcopy(net0)

# Setting useful values
magnification = 10

### Creating a data structure with all the values arranged in order

In [16]:
# Creating a data structure 
position_mat = np.ndarray(sel_mat.shape, dtype=object)

# Looping over and calculating values
for li in range(0, sel_mat.shape[0]):
    for lj in range(0, sel_mat.shape[1]):
        # Creating an empty dither dictionary
        dither_mat = {}

        # Calculating the dither values
        tmp_mat = dit.dither_16x(net0[li,lj].flatten(), 0.0125)

        # Reshaping the read dithered values to correct dimensions
        dither_mat['y'] = np.array(tmp_mat[0], dtype=object).reshape(10,10)*mm
        dither_mat['x'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm
        dither_mat['dim'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm
        
        # Initializing empty dictionaries
        pd_ideal_pos, pd_real_pos, led_mask_shift_mat, led_mask_shifted = {}, {}, {}, {}

        # Ideal position matrix for photodiodes
        ideal_start_pos, ideal_end_pos, ideal_sep = 1.25*mm, 25*mm, 2.5*mm
        pd_ideal_arr = np.arange(ideal_start_pos, ideal_end_pos, ideal_sep)
        pd_ideal_pos['x'], pd_ideal_pos['y'] =  np.meshgrid(pd_ideal_arr, pd_ideal_arr)
        pd_ideal_pos['x'], pd_ideal_pos['y'] = convert_coords(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])

        # Real photodiode positions
        pd_real_pos['x'], pd_real_pos['y'] = PD_dev1_cen['x'], PD_dev1_cen['y']
        pd_real_pos['x'], pd_real_pos['y'] = convert_coords(25*mm, PD_dev1_cen['x'], PD_dev1_cen['y'])

        # Calculating the shift
        led_mask_shift_mat['x'], led_mask_shift_mat['y'] = (1/magnification) * (pd_ideal_pos['x'] - pd_real_pos['x']), (1/magnification) * (pd_ideal_pos['y'] - pd_real_pos['y']) 

        # Creating empty placeholder matrices
        led_mask_shifted['x'] = np.zeros(dither_mat['x'].shape, dtype=object)
        led_mask_shifted['y'] = np.zeros(dither_mat['y'].shape, dtype=object)
        led_mask_shifted['dim'] = dither_mat['dim']

        # Calculating the final positions
        for lk in range(0, dither_mat['x'].shape[0]):
            for ll in range(0, dither_mat['y'].shape[1]):
                led_mask_shifted['x'][lk,ll] = dither_mat['x'][lk,ll] + led_mask_shift_mat['x'][lk,ll]
                led_mask_shifted['y'][lk,ll] = dither_mat['y'][lk,ll] + led_mask_shift_mat['y'][lk,ll]
        
        # Assigning the calculated dictionaries in the data structure
        position_mat[li,lj] = led_mask_shifted
        # print("li: {} \t lj: {}".format(str(li),str(lj)))
        

ValueError: cannot reshape array of size 0 into shape (10,10)

### Creating shift matrices for the dither blocks 
The shift of the psotion of each of the individual dither blocks depends on the magnifications and the shift of the photodidoes from their ideal position. The way this can be calculated is as follows
\begin{align*}
  x_{\textrm{shifted}} &= x + \frac{1}{\textrm{mag}}\big(x^{pd}_{\textrm{real}} - x^{pd}_{\textrm{ideal}}\big) \\
  y_{\textrm{shifted}} &= y + \frac{1}{\textrm{mag}}\big(y^{pd}_{\textrm{real}} - y^{pd}_{\textrm{ideal}}\big)
\end{align*}

In [13]:
# Initializing empty dictionaries
pd_ideal_pos, pd_real_pos, led_mask_shift_mat, led_mask_shifted = {}, {}, {}, {}

# Setting useful values
magnification = 10

# Ideal position matrix for photodiodes
ideal_start_pos, ideal_end_pos, ideal_sep = 1.25*mm, 25*mm, 2.5*mm
pd_ideal_arr = np.arange(ideal_start_pos, ideal_end_pos, ideal_sep)
pd_ideal_pos['x'], pd_ideal_pos['y'] =  np.meshgrid(pd_ideal_arr, pd_ideal_arr)
pd_ideal_pos['x'], pd_ideal_pos['y'] = convert_coords(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])

# Real photodiode positions
pd_real_pos['x'], pd_real_pos['y'] = PD_dev1_cen['x'], PD_dev1_cen['y']
pd_real_pos['x'], pd_real_pos['y'] = convert_coords(25*mm, PD_dev1_cen['x'], PD_dev1_cen['y'])

# Calculating the shift
led_mask_shift_mat['x'], led_mask_shift_mat['y'] = (1/magnification) * (pd_ideal_pos['x'] - pd_real_pos['x']), (1/magnification) * (pd_ideal_pos['y'] - pd_real_pos['y']) 

# Creating empty placeholder matrices
led_mask_shifted['x'] = np.zeros(dither_mat['x'].shape, dtype=object)
led_mask_shifted['y'] = np.zeros(dither_mat['y'].shape, dtype=object)
led_mask_shifted['dim'] = dither_mat['dim']

# Calculating the final positions
for li in range(0, dither_mat['x'].shape[0]):
    for lj in range(0, dither_mat['y'].shape[1]):
        led_mask_shifted['x'][li,lj] = dither_mat['x'][li,lj] + led_mask_shift_mat['x'][li,lj]
        led_mask_shifted['y'][li,lj] = dither_mat['y'][li,lj] + led_mask_shift_mat['y'][li,lj]

In [14]:
led_mask_shifted['x'][1]
temp_real = pd_real_pos['x']
temp_ideal = pd_ideal_pos["x"]

### Creating position matrix for the larger block

### Calculating the shifted positions according to the real LED position

In [17]:
# Choosing the data source to follow
led_positions = copy.deepcopy(LED_input_cen)
led_positions_centered = {}

# Shifting the coordinates to the center
# led_positions_centered['x'], led_positions_centered['y'] = shift_to_center(25*mm, led_positions['x'], led_positions['y'])
led_positions_centered['x'], led_positions_centered['y'] = led_positions['x'], led_positions['y']
# Calclating the shift from the center
led_positions_shifts = {}
led_positions_shifts['x'], led_positions_shifts['y'] = (1/magnification) * (np.zeros(led_positions['x'].shape) - led_positions_centered['x']), (1/magnification) * (np.zeros(led_positions['y'].shape) - led_positions_centered['y'])

# Shifting all the values 
for li in range(0,led_positions['x'].shape[0]):
    for lj in range(0, led_positions['y'].shape[1]):
        position_mat[li,lj]['x'] = position_mat[li,lj]['x'] + led_positions_centered['x'][li,lj] + led_positions_shifts['x'][li,lj]
        position_mat[li,lj]['y'] = position_mat[li,lj]['y'] + led_positions_centered['y'][li,lj] + led_positions_shifts['y'][li,lj]
position_mat_flat = position_mat.flatten()

TypeError: 'NoneType' object is not subscriptable

### Creating a test file with squares positioned at the coordinate values

In [60]:
for idx1 in range(0, position_mat_flat.shape[0]):
    for idx2 in range(0, position_mat_flat[idx1]['x'].shape[0]):
        for idx3 in range(0, position_mat_flat[idx1]['x'].shape[0]):
            selected_dict = {}
            selected_dict['x'], selected_dict['y'], selected_dict['dim'] = position_mat_flat[idx1]['x'][idx2,idx3], position_mat_flat[idx1]['y'][idx2,idx3], position_mat_flat[idx1]['y'][idx2,idx3] 
            if selected_dict['x'].size == 0 or selected_dict['y'].size == 0:
                 continue
            for idx4 in range(0, selected_dict['x'].shape[0]):
                pattern = gdstk.rectangle(*utils.convert_rect([selected_dict['dim'][idx4], selected_dict['dim'][idx4]], [selected_dict['x'][idx4], selected_dict['y'][idx4]]))
                masks.add(pattern)
                

In [61]:
temp_x = position_mat[0,0]['x']

### Saving the GDS file

In [62]:
mask_array.write_gds('test.gds')